In [14]:
import os
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# Define constants
IMG_HEIGHT, IMG_WIDTH = 150, 150
BATCH_SIZE = 32
EPOCHS = 15

# Print current working directory and list contents of data directory
print("Current working directory:", os.getcwd())
print("Contents of data directory:", os.listdir('data'))

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

def process_path(file_path):
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    # Get the label from the file path
    label = tf.strings.split(file_path, os.path.sep)[-2] == 'cage'
    return img, label

def prepare_dataset(data_dir):
    # Get list of files in directory
    file_list = tf.data.Dataset.list_files(str(data_dir + '/*/*'))
    # Shuffle the files
    file_list = file_list.shuffle(buffer_size=10000)
    # Map the process_path function to each file path
    labeled_ds = file_list.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    return labeled_ds

# Prepare the datasets
train_ds = prepare_dataset('data')
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Building the CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds,
    epochs=EPOCHS
)

# Save the model
model.save('nicholas_cage_detector.h5')
print("Model training completed and saved as 'nicholas_cage_detector.h5'")

# Function to make predictions (for future use)
def predict_image(model, img_path):
    img = tf.io.read_file(img_path)
    img = decode_img(img)
    img = tf.expand_dims(img, 0) / 255.0
    
    prediction = model.predict(img)
    return prediction[0][0]

print("Script completed successfully.")

TensorFlow version: 2.17.0
Current working directory: c:\Users\keega\OneDrive\Documents\Python\VEs\PDAN_ICE
Contents of data directory: ['cage', 'others']
Epoch 1/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 266ms/step - accuracy: 0.5234 - loss: 721.9409
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step - accuracy: 0.5256 - loss: 33.4330
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step - accuracy: 0.5102 - loss: 8.2402
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - accuracy: 0.5467 - loss: 1.3613
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step - accuracy: 0.6783 - loss: 0.8027
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step - accuracy: 0.8193 - loss: 0.4951
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.8434 - loss: 0.3355
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.9828 - loss: 0.0866
Epoch 9/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step - accuracy: 1.0000 - loss: 0.0245
Epoch 10/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 1.0000 - loss: 0.0156
E

Model training completed and saved as 'nicholas_cage_detector.h5'
Script completed successfully.


In [15]:
import tensorflow as tf
import os

# Load the saved model
model = tf.keras.models.load_model('nicholas_cage_detector.h5')

# Function to decode and preprocess the image
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [150, 150])

# Function to predict on a single image
def predict_image(img_path):
    # Load the raw data from the file as a string
    img = tf.io.read_file(img_path)
    img = decode_img(img)
    img = tf.expand_dims(img, 0) / 255.0
    
    prediction = model.predict(img)
    return prediction[0][0]

# Test directory
test_dir = 'test'

# Iterate over all images in the test directory
for img_name in os.listdir(test_dir):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(test_dir, img_name)
        result = predict_image(img_path)
        print(f"Image: {img_name}")
        print(f"Probability of being Nicholas Cage: {result:.2f}")
        print("--------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Image: images_1.jpeg
Probability of being Nicholas Cage: 0.51
--------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Image: images_2.jpeg
Probability of being Nicholas Cage: 0.52
--------------------


I possibly need more training data for this.